### **Importing Libraries**

In [1]:
# Importing the necessary libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

from torch.optim.lr_scheduler import ReduceLROnPlateau, ExponentialLR, CyclicLR, OneCycleLR, StepLR

import numpy as np
import random

from datetime import datetime
from pathlib import Path
import sys
from types import SimpleNamespace

from IPython.display import Image
import matplotlib.pyplot as plt
%matplotlib inline

import wandb

### **Set seed**

In [2]:
SEED = 2345
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### **Wanbd Login**

In [3]:
# Login to W&B
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: piyushrs. Use `wandb login --relogin` to force relogin


True

### **Data Folder declaration**

In [4]:
base_folder = Path('u:\\OneDrive - The University of Texas at Dallas\\6382\\Datasets')

In [5]:
data_folder = base_folder/'data'
archive_folder = base_folder/'archive'
model_folder = base_folder/'models/CIFAR_10'
custom_functions = base_folder/'custom-functions'

In [6]:
model_folder.mkdir(exist_ok=True, parents=True)
data_folder.mkdir(exist_ok=True, parents=True)

In [7]:
sys.path.append(str(model_folder))
sys.path

['u:\\OneDrive - The University of Texas at Dallas\\6382\\Assignment 6',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310\\DLLs',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310\\lib',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310',
 '',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\Pythonwin',
 'u:\\OneDrive - The University of Texas at Dallas\\6382\\Datasets\\models\\CIFAR_10']

### **Transformations and downloading dataset**

In [8]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

valid_transform = transforms.Compose([                             
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Download the training_validation data (we will create two subsets - trainset and valset frpm this)
train_set = torchvision.datasets.CIFAR10(root = data_folder, 
                                             train = True, 
                                             transform = train_transform, 
                                             download = True)

valid_set = torchvision.datasets.CIFAR10(root = data_folder, 
                                             train = True, 
                                             transform = valid_transform, 
                                             download = True)

# Download the testing data
testset = torchvision.datasets.CIFAR10(root = data_folder, 
                                            train = False, 
                                            transform = valid_transform, 
                                            download = True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


### **Split dataset function**

In [9]:
def split_dataset(base_dataset, fraction, seed):
    split_a_size = int(fraction * len(base_dataset))
    split_b_size = len(base_dataset) - split_a_size
    return torch.utils.data.random_split(base_dataset, [split_a_size, split_b_size], 
                                         generator=torch.Generator().manual_seed(seed)
    )

In [10]:
trainset, _ = split_dataset(train_set, 0.8, 42)
_, validset = split_dataset(valid_set, 0.8, 42)

In [11]:
len(trainset), len(validset)

(40000, 10000)

In [12]:
trainset.dataset.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [13]:
trainset.dataset.data.shape

(50000, 32, 32, 3)

### <font color = "aqua"> **Custom CNN**

In [14]:
class CustomCNN_Best(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3,3), padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=(3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=(3,3), padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=(3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=(3,3), padding=1),
            nn.ReLU(), 
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, kernel_size=(3,3), padding=1),
            nn.ReLU(), 
            nn.MaxPool2d(2)
        )
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(4096, 1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

In [15]:
class CIFAR10(nn.Module):
    
    def __init__(self):
        super().__init__()
        super(CIFAR10, self).__init__()

        self.conv1_layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding='same'), 
            nn.LeakyReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding='same'), 
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride = 2 ) 
        )

        self.conv2_layer = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding='same'), 
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding='same'), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2) 
        )

        self.conv3_layer = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding='same'), 
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding='same'), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2) 
        )

        self.flatten = nn.Flatten()      
        self.fc1 = nn.Linear(16384, out_features=1024)      
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

        self.drop1 = nn.Dropout(0.5)
        self.drop2 = nn.Dropout(0.5)
      
        
    def forward(self, x):
    # conv layers
        out = self.conv1_layer(x)
        # out = self.conv2_layer(out)
        # out = self.conv3_layer(out)

        # flatten befrore input to linear layer
        out = self.flatten(out)

        # linear hidden layers
        out = F.relu(self.fc1(out))
        out = self.drop1(out)
        out = F.relu(self.fc2(out))
        out = self.drop2(out)

        # output layer - no softmax as it is applied by nn.CrossEntropyLoss

        out = self.fc3(out)

        return out

In [16]:
class ConvNet(torch.nn.Module):

    def __init__(self):
        super().__init__()
        
        #########################
        ### 1st residual block
        #########################
        
        self.block_1 = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=3,
                                out_channels=6,
                                kernel_size=(1, 1),
                                stride=(1, 1),
                                padding=0),
                torch.nn.BatchNorm2d(6),
                torch.nn.ReLU(inplace=True),
                torch.nn.Conv2d(in_channels=6,
                                out_channels=3,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.BatchNorm2d(3)
        )
        
        self.block_2 = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=3,
                                out_channels=6,
                                kernel_size=(1, 1),
                                stride=(1, 1),
                                padding=0),
                torch.nn.BatchNorm2d(6),
                torch.nn.ReLU(inplace=True),
                torch.nn.Conv2d(in_channels=6,
                                out_channels=3,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.BatchNorm2d(3)
        )

        #########################
        ### Fully connected
        #########################        
        self.linear_1 = torch.nn.Linear(3*32*32, 10)

        
    def forward(self, x):
        
        #########################
        ### 1st residual block
        #########################
        shortcut = x
        x = self.block_1(x)
        x = torch.nn.functional.relu(x + shortcut)
        
        #########################
        ### 2nd residual block
        #########################
        shortcut = x
        x = self.block_2(x)
        x = torch.nn.functional.relu(x + shortcut)
        
        #########################
        ### Fully connected
        #########################
        logits = self.linear_1(x.view(-1, 3*32*32))
        return logits

In [17]:
class CustomCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size= 3, padding= 1),               # input => 3 x 32 x 32 output => 32 x 32 x 32
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace= True),
            nn.Conv2d(32, 32, kernel_size= 3, padding=1, stride= 1),        # 32 x 32 x 32
            nn.Dropout2d(0.5),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace= True),

            nn.Conv2d(32, 64, kernel_size= 3, padding= 1, stride= 1),           # 64 x 32 x 32
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace= True),
            nn.Conv2d(64, 32, kernel_size= 5, padding=2, stride=1),             # 32 x 32 x 32
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32*16*16, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

### <font color = "aqua"> **CustomCNN torch summary**

In [18]:
from torchsummary import summary
summary(CustomCNN_Best().cuda(), (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
       BatchNorm2d-3           [-1, 64, 32, 32]             128
            Conv2d-4           [-1, 64, 32, 32]          36,928
              ReLU-5           [-1, 64, 32, 32]               0
         MaxPool2d-6           [-1, 64, 16, 16]               0
            Conv2d-7          [-1, 128, 16, 16]          73,856
              ReLU-8          [-1, 128, 16, 16]               0
       BatchNorm2d-9          [-1, 128, 16, 16]             256
           Conv2d-10          [-1, 128, 16, 16]         147,584
             ReLU-11          [-1, 128, 16, 16]               0
        MaxPool2d-12            [-1, 128, 8, 8]               0
           Conv2d-13            [-1, 256, 8, 8]         295,168
             ReLU-14            [-1, 25

### **Train function**

In [19]:
def train(train_loader, loss_function, model, optimizer, device, grad_clipping, max_norm, log_batch, log_interval):

  # Training Loop 

  # initilalize variables as global
  # these counts will be updated every epoch
  global batch_ct_train

  # Initialize train_loss at the he start of the epoch
  running_train_loss = 0
  running_train_correct = 0
  
  # put the model in training mode

  model.train()
  # Iterate on batches from the dataset using train_loader
  for input_, targets in train_loader:
    
    # move inputs and outputs to GPUs
    input_ = input_.to(device)
    targets = targets.to(device)


    # Step 1: Forward Pass: Compute model's predictions 
    output = model(input_)
    
    # Step 2: Compute loss
    loss = loss_function(output, targets)

    # Correct prediction
    y_pred = torch.argmax(output, dim = 1)
    correct = torch.sum(y_pred == targets)

    batch_ct_train += 1

    # Step 3: Backward pass -Compute the gradients
    optimizer.zero_grad()
    loss.backward()

    # Gradient Clipping
    if grad_clipping:
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm, norm_type=2)

    # Step 4: Update the parameters
    optimizer.step()
          
    # Add train loss of a batch 
    running_train_loss += loss.item()

    # Add Corect counts of a batch
    running_train_correct += correct

    # log batch loss and accuracy
    if log_batch:
      if ((batch_ct_train + 1) % log_interval) == 0:
        wandb.log({f"Train Batch Loss  :": loss})
        wandb.log({f"Train Batch Acc :": correct/len(targets)})

  
  # Calculate mean train loss for the whole dataset for a particular epoch
  train_loss = running_train_loss/len(train_loader)

  # Calculate accuracy for the whole dataset for a particular epoch
  train_acc = running_train_correct/len(train_loader.dataset)
  

  return train_loss, train_acc

### **Validate Function**

In [20]:
def validate(valid_loader, loss_function, model, device, log_batch, log_interval):

  # initilalize variables as global
  # these counts will be updated every epoch
  global batch_ct_valid

  # Validation/Test loop
  # Initialize valid_loss at the he strat of the epoch
  running_val_loss = 0
  running_val_correct = 0

  # put the model in evaluation mode
  model.eval()

  with torch.no_grad():
    for input_,targets in valid_loader:

      # move inputs and outputs to GPUs
      input_ = input_.to(device)
      targets = targets.to(device)

      # Step 1: Forward Pass: Compute model's predictions 
      output = model(input_)

      # Step 2: Compute loss
      loss = loss_function(output, targets)

      # Correct Predictions
      y_pred = torch.argmax(output, dim = 1)
      correct = torch.sum(y_pred == targets)

      batch_ct_valid += 1

      # Add val loss of a batch 
      running_val_loss += loss.item()

      # Add correct count for each batch
      running_val_correct += correct

      # log batch loss and accuracy
      if log_batch:
        if ((batch_ct_valid + 1) % log_interval) == 0:
          wandb.log({f"Valid Batch Loss  :": loss})
          wandb.log({f"Valid Batch Accuracy :": correct/len(targets)})

    # Calculate mean val loss for the whole dataset for a particular epoch
    val_loss = running_val_loss/len(valid_loader)

    # Calculate accuracy for the whole dataset for a particular epoch
    val_acc = running_val_correct/len(valid_loader.dataset)

    # scheduler step
    # scheduler.step(val_loss)
    # scheduler.step()
    
  return val_loss, val_acc

### **Train Loop Function**

In [21]:
def train_loop(train_loader, valid_loader, model, optimizer, loss_function, epochs, device, patience, early_stopping,
               file_model, save_best_model):
    
  """ 
  Function for training the model and plotting the graph for train & validation loss vs epoch.
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate, batch size.
  Output: final weights, bias and train loss and validation loss for each epoch.
  """

  # Create lists to store train and val loss at each epoch
  train_loss_history = []
  valid_loss_history = []
  train_acc_history = []
  valid_acc_history = []

  # initialize variables for early stopping

  delta = 0
  best_score = None
  valid_loss_min = np.Inf
  counter_early_stop=0
  early_stop=False

  # Iterate for the given number of epochs
  # Step 5: Repeat steps 1 - 4

  for epoch in range(epochs):

    t0 = datetime.now()

    # Get train loss and accuracy for one epoch
    train_loss, train_acc = train(train_loader, loss_function, model, optimizer,
                                  wandb.config.device, wandb.config.grad_clipping, 
                                  wandb.config.max_norm, wandb.config.log_batch, wandb.config.log_interval)
    valid_loss, valid_acc   = validate(valid_loader, loss_function, model, wandb.config.device, wandb.config.log_batch, wandb.config.log_interval)

    dt = datetime.now() - t0

    # Save history of the Losses and accuracy
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)

    valid_loss_history.append(valid_loss)
    valid_acc_history.append(valid_acc)

    # Log the train and valid loss to wandb
    wandb.log({f"Train Loss :": train_loss, "epoch": epoch})
    wandb.log({f"Train Acc :": train_acc, "epoch": epoch})

    wandb.log({f"Valid Loss :": valid_loss, "epoch": epoch})
    wandb.log({f"Valid Acc :": valid_acc, "epoch": epoch})

    if early_stopping:
      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        counter_early_stop += 1
        print(f'Early stoping counter: {counter_early_stop} out of {patience}')
        if counter_early_stop > patience:
          early_stop = True

      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        counter_early_stop=0
        valid_loss_min = valid_loss

      if early_stop:
        print('Early Stopping')
        break

    elif save_best_model:

      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        print(f'Validation loss has not decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Not Saving Model...')
      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss
        
    else:
        torch.save(model.state_dict(), file_model)
    
    # Print the train loss and accuracy for given number of epochs, batch size and number of samples
    print(f'Epoch : {epoch+1} / {epochs}')
    print(f'Time to complete {epoch+1} is {dt}')
    # print(f'Learning rate: {scheduler._last_lr[0]}')
    print(f'Train Loss: {train_loss : .4f} | Train Accuracy: {train_acc * 100 : .4f}%')
    print(f'Valid Loss: {valid_loss : .4f} | Valid Accuracy: {valid_acc * 100 : .4f}%')
    print()
    torch.cuda.empty_cache()

  return train_loss_history, train_acc_history, valid_loss_history, valid_acc_history

### **Get accuracy prediction function**

In [22]:
def get_acc_pred(data_loader, model, device):
    
  """ 
  Function to get predictions and accuracy for a given data using estimated model
  Input: Data iterator, Final estimated weoights, bias
  Output: Prections and Accuracy for given dataset
  """

  # Array to store predicted labels
  predictions = torch.Tensor() # empty tensor
  predictions = predictions.to(device) # move predictions to GPU

  # Array to store actual labels
  y = torch.Tensor() # empty tensor
  y = y.to(device)

  # put the model in evaluation mode
  model.eval()
  
  # Iterate over batches from data iterator
  with torch.no_grad():
    for input_, targets in data_loader:
      
      # move inputs and outputs to GPUs
      
      input_ = input_.to(device)
      targets = targets.to(device)
      
      # Calculated the predicted labels
      output = model(input_)

      # Choose the label with maximum probability
      prediction = torch.argmax(output, dim = 1)

      # Add the predicted labels to the array
      predictions = torch.cat((predictions, prediction)) 

      # Add the actual labels to the array
      y = torch.cat((y, targets)) 

  # Check for complete dataset if actual and predicted labels are same or not
  # Calculate accuracy
  acc = (predictions == y).float().mean()

  # Return tuple containing predictions and accuracy
  return predictions, acc  

### **Meta Data**

In [88]:
hyperparameters = SimpleNamespace(
    epochs = 10,
    output_dim = 10,
    batch_size= 256,
    learning_rate=0.005,
    dataset="CIFAR10",
    architecture="CustomCNN",
    log_interval = 25,
    log_batch = True,
    file_model = model_folder/'assignment_partB_customCNN_CIFAR10.pt',
    grad_clipping = False,
    max_norm = 1,
    momentum = 0.9,
    patience = 5,
    early_stopping = False,
    scheduler_factor = 0,
    scheduler_patience = 0,
    weight_decay = 0.00001,
    save_best_model = True,
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
)

In [89]:
wandb.init(name = "exp13", project = 'CustomCNN11_HW6_B' , config = hyperparameters)

In [90]:
wandb.config

{'epochs': 10, 'output_dim': 10, 'batch_size': 256, 'learning_rate': 0.005, 'dataset': 'CIFAR10', 'architecture': 'CustomCNN', 'log_interval': 25, 'log_batch': True, 'file_model': 'u:\\OneDrive - The University of Texas at Dallas\\6382\\Datasets\\models\\CIFAR_10\\assignment_partB_customCNN_CIFAR10.pt', 'grad_clipping': False, 'max_norm': 1, 'momentum': 0.9, 'patience': 5, 'early_stopping': False, 'scheduler_factor': 0, 'scheduler_patience': 0, 'weight_decay': 1e-05, 'save_best_model': True, 'device': 'cuda:0'}

### **Data Loaders**

In [26]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=wandb.config.batch_size, shuffle = True)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=wandb.config.batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(testset, batch_size=wandb.config.batch_size, shuffle=False)

### **Init weights with Kaiming initialization**

In [27]:
def init_weight(layer):
  if type(layer) == nn.Linear:
    torch.nn.init.kaiming_normal_(layer.weight)
    torch.nn.init.zeros_(layer.bias)
  
  if type(layer) == nn.Conv2d:
    torch.nn.init.kaiming_normal_(layer.weight)
    torch.nn.init.zeros_(layer.bias)

In [28]:
def init_weight1(layer):
  if type(layer) == nn.Linear:
    torch.nn.init.xavier_normal_(layer.weight)
    torch.nn.init.zeros_(layer.bias)
  
  if type(layer) == nn.Conv2d:
    torch.nn.init.xavier_normal_(layer.weight)
    torch.nn.init.zeros_(layer.bias)

In [91]:
model = CustomCNN_Best()
model.to(wandb.config.device)
# model.apply(init_weight)
loss_function = nn.CrossEntropyLoss()
loss_function.to(wandb.config.device)
# optim = torch.optim.Adam(model.parameters(), lr = wandb.config.learning_rate, weight_decay=0.0001)
# scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size= 30, gamma=0.1)
# optim = torch.optim.Adagrad(model.parameters(), lr=wandb.config.learning_rate, weight_decay= 0.00001)
# optim = torch.optim.RMSprop(model.parameters(), lr=wandb.config.learning_rate, momentum= wandb.config.momentum, weight_decay= wandb.config.weight_decay)
optim = torch.optim.SGD(model.parameters(), lr = wandb.config.learning_rate, momentum= wandb.config.momentum, weight_decay= wandb.config.weight_decay, nesterov= False)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optim, max_lr=0.15, total_steps=len(train_loader) * wandb.config.epochs, epochs=wandb.config.epochs, three_phase=True)

# scheduler = ReduceLROnPlateau(optim, mode='min', factor= wandb.config.scheduler_factor, patience=wandb.config.scheduler_patience, verbose=True)

wandb.watch(model)

[]

### <font color = "Red"> **Training Model**

In [92]:
# See live graphs in the notebook.
#%%wandb 
batch_ct_train, batch_ct_valid = 0, 0
train_loss_history, train_acc_history, valid_loss_history, valid_acc_history = train_loop(train_loader, 
                                                                                          valid_loader, 
                                                                                          model, 
                                                                                          optim, 
                                                                                          loss_function, 
                                                                                          wandb.config.epochs, 
                                                                                          wandb.config.device,
                                                                                          wandb.config.patience,
                                                                                          wandb.config.early_stopping,
                                                                                          wandb.config.file_model,
                                                                                          wandb.config.save_best_model
                                                                                          )

Validation loss has decreased (inf --> 1.117806). Saving Model...
Epoch : 1 / 10
Time to complete 1 is 0:00:22.927882
Train Loss:  1.5089 | Train Accuracy:  44.1750%
Valid Loss:  1.1178 | Valid Accuracy:  59.6200%

Validation loss has decreased (1.117806 --> 0.847781). Saving model...
Epoch : 2 / 10
Time to complete 2 is 0:00:22.009028
Train Loss:  0.9682 | Train Accuracy:  65.5800%
Valid Loss:  0.8478 | Valid Accuracy:  69.7500%

Validation loss has decreased (0.847781 --> 0.780075). Saving model...
Epoch : 3 / 10
Time to complete 3 is 0:00:21.765638
Train Loss:  0.7506 | Train Accuracy:  73.6425%
Valid Loss:  0.7801 | Valid Accuracy:  72.5300%

Validation loss has decreased (0.780075 --> 0.732226). Saving model...
Epoch : 4 / 10
Time to complete 4 is 0:00:21.811886
Train Loss:  0.6191 | Train Accuracy:  78.0225%
Valid Loss:  0.7322 | Valid Accuracy:  74.3400%

Validation loss has decreased (0.732226 --> 0.673304). Saving model...
Epoch : 5 / 10
Time to complete 5 is 0:00:22.350078
Tr

In [93]:
wandb.finish()

Train Acc :,▁▄▅▆▆▇▇▇██
Train Batch Acc :,▁▂▃▄▅▅▅▅▅▆▆▆▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇█▇█▇████████
Train Batch Loss :,█▇▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▁▂▂▁▂▁▂▁▁▁▁▁▁▁▁
Train Loss :,█▅▄▃▃▂▂▂▁▁
Valid Acc :,▁▄▅▆▆▇▇▇██
Valid Batch Accuracy :,▁▂▂▃▅▆▅▅▆▇▅▅▆█▇▇
Valid Batch Loss :,█▆▇▆▄▃▄▄▂▁▄▇▅▁▄▂
Valid Loss :,█▄▃▃▂▁▂▂▂▂
epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
Train Acc :,0.95127
Train Batch Acc :,0.91406
